In [2]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
net = torchvision.models.mobilenet_v2(pretrained=False)
num_ftrs = net.last_channel

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net.classifier = nn.Linear(num_ftrs, len(classes))
net.cuda()
print()

In [5]:
experiment_note = """ 
NET: mobilenet_v2
Expereiment goal: collect the data
Net status: False
Training mode: Normal Evo + Crossn
Optimizer SGD
lr=0.001 
momentum=0.9
epochs= 30
Params: Default"""

logger = Logger('train', experiment_note)

In [6]:
logger.path

'./experiments/2019-11-25/train_21:18'

In [8]:
# post_note = "Test After Experiment info"
# logger.add_post_result(post_note)

In [7]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
evo_optim = CrossN()

def validation(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return float(100 * correct / total)

In [8]:
solver = Solver(
    net,
    optimizer,
    logger,
    criterion,
    validation,
    evo_optim, 
    trainloader,
    testloader,
    testloader,
    mode='normal')

In [ ]:
solver.start()

Start training
Epoch: 0


100%|██████████| 19/19 [00:30<00:00,  1.62s/it]

best child - 10.0
Epoch: 1


[2] loss: 2.207 validation score: 16.22 %
Epoch: 2
[3] loss: 2.067 validation score: 23.25 %
Epoch: 3
[4] loss: 1.913 validation score: 29.07 %
Epoch: 4
[5] loss: 1.721 validation score: 33.06 %
Epoch: 5


100%|██████████| 19/19 [00:30<00:00,  1.61s/it]


best child - 32.94
Epoch: 6
[7] loss: 1.669 validation score: 36.16 %
Epoch: 7
[8] loss: 1.636 validation score: 37.88 %
Epoch: 8
[9] loss: 1.538 validation score: 39.85 %
Epoch: 9
[10] loss: 1.483 validation score: 41.42 %
Epoch: 10


  5%|▌         | 1/19 [00:01<00:26,  1.48s/it]